# Write Python code to summarize the project cost

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime 
import os

Input data files file path in the current dictonary and load data to dataframe.
Importing Project Cost data and drop the na and columns ('sr_no', 'SubSection','Project_Code')

In [2]:
file_path ={
    'htf_stf':r"data\HTF_STF_30_07_2024.xlsx",
    'cost_code':r'data\RSMLI Cost code.xlsx'
    }
file_path

{'htf_stf': 'data\\HTF_STF_30_07_2024.xlsx',
 'cost_code': 'data\\RSMLI Cost code.xlsx'}

In [3]:
df_cost_code = pd.read_excel(file_path['cost_code'],sheet_name='Costcode',index_col='sr_no')
df_cost_code = df_cost_code.dropna(subset=['Section']).reset_index().drop(['SubSection','Project_Code'], axis=1)

# Extract HTF information from HTF file under htf sheet and store it to dataframe
htf_columns = ['Project','Status','Service Date',
               'HTF no', 'HTF Date','Line No',
               'Item number','Açıklama / Kullanım Yeri','HTF Qty','Unit_en',
               'Currency','Estimated unit price','Estimate amount']

df_htf = pd.read_excel(file_path['htf_stf'], sheet_name="HTF", usecols=htf_columns)

# Data general information

In [4]:
df_htf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3895 entries, 0 to 3894
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   HTF no                    3895 non-null   object        
 1   HTF Date                  2758 non-null   datetime64[ns]
 2   Line No                   3895 non-null   int64         
 3   Project                   3895 non-null   object        
 4   Status                    3895 non-null   object        
 5   Service Date              3895 non-null   datetime64[ns]
 6   Item number               3895 non-null   object        
 7   Açıklama / Kullanım Yeri  3883 non-null   object        
 8   HTF Qty                   3895 non-null   float64       
 9   Estimated unit price      3895 non-null   float64       
 10  Estimate amount           3895 non-null   float64       
 11  Currency                  3895 non-null   object        
 12  Unit_en             

In [5]:
df_htf.columns

Index(['HTF no', 'HTF Date', 'Line No', 'Project', 'Status', 'Service Date',
       'Item number', 'Açıklama / Kullanım Yeri', 'HTF Qty',
       'Estimated unit price', 'Estimate amount', 'Currency', 'Unit_en'],
      dtype='object')

In [6]:
total_estimate_amount = df_htf[df_htf['Status']=='Invoiced']['Estimate amount'].sum()
print(f'Total Estimate Amount for the Invoced Status of HTDFs: SAR {total_estimate_amount:,.2f}')

Total Estimate Amount for the Invoced Status of HTDFs: SAR 89,714,127.19


In [7]:
df_cost_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359 entries, 0 to 358
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   sr_no                     359 non-null    int64 
 1   Project                   359 non-null    object
 2   Project (Cost-Code) Name  359 non-null    object
 3   Category                  359 non-null    object
 4   Category_Name             359 non-null    object
 5   Section                   359 non-null    object
 6   Section_name              359 non-null    object
dtypes: int64(1), object(6)
memory usage: 19.8+ KB


In [8]:
df_cost_code.columns

Index(['sr_no', 'Project', 'Project (Cost-Code) Name', 'Category',
       'Category_Name', 'Section', 'Section_name'],
      dtype='object')

In [10]:
# Cost Code Categories
# print("Code  Category")
# print('----------------------')
# for cost_code in df_cost_code['Category'].unique():
#     category_name = df_cost_code[df_cost_code['Category']== cost_code ]['Project (Cost-Code) Name'].iloc[0]
#     print(cost_code,' ',category_name)

In [61]:
# Cost Code Section
# print("#  Code  Section")
# print('-------------------------------------')
# for i, code in enumerate(df_cost_code['Section'].dropna().unique()):
#     section_name = df_cost_code[df_cost_code['Section']== code ]['Section_name'].iloc[0]
#     print(f'{i:>2} {code:^3} {section_name}')

# HTF data process starting

In [328]:
# Filter only Status approved and Estimate amount non-zero
htf_summary = df_htf[(df_htf['Status']=="Invoiced") & (df_htf['Estimate amount']!=0)]

# Rename the Line description column
htf_summary = htf_summary.rename(columns={'Açıklama / Kullanım Yeri':'Line description'})


# add yyyy-mm column to dataframe
htf_summary['year_month'] =  pd.to_datetime(htf_summary['Service Date']).dt.strftime('%Y-%m')
# htf_summary['month'] =  pd.to_datetime(htf_summary['Service Date']).dt.strftime('%b')
# set datetime data type to the date columns0
htf_summary['Service Date'] = pd.to_datetime(htf_summary['Service Date']).dt.strftime('%Y-%m-%d')
htf_summary['HTF Date'] = pd.to_datetime(htf_summary['HTF Date']).dt.strftime('%Y-%m-%d')

# group the dataframe based year_month and Project (cost code)
htf_summary = htf_summary.groupby(['year_month','Project'])[['Estimate amount']].sum()

# display the formated Dataframe
htf_summary.head(5).style.set_caption("RSMLI HTF - SUMMARZIED : BY YEAR-MONTH").format(precision = 2, thousands = ',')

In [327]:
htf_pivot = htf_summary.reset_index().pivot_table(
        values='Estimate amount'
        ,index=['Project']
        ,columns=['year_month']
        # ,aggfunc=sum
        ,margins= True
        ,margins_name="Total"
        ,sort= 'False'
).sort_index(axis = 1, level = 'year_month')

htf_pivot.head(10).style.set_caption("RSMLI HTF - MONTLY SUMMARY : BY PROJECT COST CODE").format(precision= 2, thousands= ',')

year_month,2023-01,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,Total
Project,,,,,,,,,,,,,,,,,,,
P.CSE.CMS.CIARI,nan,nan,nan,nan,nan,nan,nan,nan,nan,"1,392.51",nan,nan,nan,575.04,nan,nan,nan,nan,983.77
P.CSE.CMS.WRKSH,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,275.00,nan,nan,nan,nan,nan,nan,275.00
P.CSE.DRM,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"80,974.95",nan,nan,nan,"80,974.95"
P.CSE.TOL.CIARI,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"9,878.50","9,878.50"
P.CSE.TOL.LABTS,nan,nan,nan,"2,685.00",nan,nan,nan,nan,"2,918.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,"2,801.50"
P.CSE.TST,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"652,464.00","652,464.00"
P.D01.001,nan,nan,nan,nan,nan,nan,nan,nan,nan,"4,600.00","15,200.00",nan,nan,"9,056.00",nan,nan,nan,nan,"9,618.67"
P.D01.004,nan,nan,nan,nan,"1,677,446.35",nan,"3,601,403.49",nan,"2,726,017.64","496,974.02","906,695.52","358,635.52",nan,"621,271.95","596,916.39",nan,"69,656.98","182,782.52","1,123,780.04"
P.D01.005,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"10,915.00",nan,nan,nan,nan,nan,nan,"10,915.00"


In [264]:
# merge_columns = ['sr_no','Project', 'Category_Name', 'Section', 'Section_name']
# htf_pivot.merge(df_cost_code[merge_columns],on='Project',how = 'left').set_index(merge_columns).reset_index()

In [329]:
# Summarize the estimate amount by month and add total row at bottoms
htf_monthly = htf_summary.groupby(['year_month']).sum()
htf_monthly.loc['Total'] =htf_monthly.sum(axis = 0)

htf_monthly.style.set_caption("RSMLI HTF MONTHLY SUMMARIZED").format(precision = 2, thousands = ',')

,Estimate amount
year_month,
2023-01,"17,700.00"
2023-03,"136,837.00"
2023-04,"733,059.06"
2023-05,"3,031,569.14"
2023-06,"2,338,595.29"
2023-07,"709,692.87"
2023-08,"5,726,987.17"
2023-09,"5,103,755.72"
2023-10,"9,810,738.62"


In [313]:
# Filter the dataframe for the last month data and find the total amouunt
last_date = df_htf['Service Date'].max().replace(day=1)

htf_last_month = df_htf[(df_htf['Status']=="Invoiced") & (df_htf['Estimate amount']!=0) & (df_htf['Service Date'] >=last_date)]

total_last_month_amount = htf_last_month['Estimate amount'].sum()
print(f'Total Estimate Amount for the {last_date:%b-%Y} month Invoced Status of HTDFs: SAR {total_last_month_amount:,.2f}')

Total Estimate Amount for the Jul-2024 month Invoced Status of HTDFs: SAR 7,967,360.62


In [321]:

with pd.ExcelWriter ('HTF_Total.xlsx') as writer:

    # HTF summary by Category and Section
    merge_columns = ['sr_no','Project', 'Category_Name', 'Section', 'Section_name']
    (htf_pivot.merge(df_cost_code[merge_columns],on='Project',how = 'left').set_index(merge_columns)
     ).to_excel(writer,sheet_name='Section',float_format="%f", index = True,startrow = 1, startcol = 1, merge_cells = False, freeze_panes = (2,6))
    
    # HTF summary by Category only
    merge_columns = ['sr_no','Project', 'Category','Category_Name']
    (htf_pivot.merge(df_cost_code[merge_columns],on='Project',how = 'left').set_index(merge_columns)
     .groupby(by = ['Category','Category_Name'],sort = False, dropna = False ).sum()
     ).to_excel(writer,sheet_name='Category',float_format="%f", index = True,startrow = 1, startcol = 1, merge_cells = False, freeze_panes = (2,3))
    
    # HTF Monthly summary
    htf_monthly.style.format('{:.2f}').to_excel(writer,sheet_name='Monthly',float_format="%.2f", index = True,startrow = 1, startcol = 1, merge_cells = False, freeze_panes = (2,2))
    
    # HTF Last Month summary
    htf_last_month.to_excel(writer,sheet_name=last_date.strftime('%b-%Y'),float_format="%.2f", index = True,startrow = 1, startcol = 1, merge_cells = False, freeze_panes = (2,2))
        
os.startfile('HTF_Total.xlsx','edit')

# STH data process starting

In [61]:
stf_columns = ['Project',
               'STF No','Line No','Item name','Buyer group','Requester',
               'STF Date','Delivery date','Status',
               'Unit','STF qty','Received Qty','Refunded Quantity','Remaining quantity','Realized pct (%)',
               'Currency','Estimated unit price','Receipt Unit price',
               'Estimate amount','Receipt Amount','Remaining amount','Estimated Miscellaneous Expense',
               'Miscellaneous Expense Incurred','The Rest Miscellaneous Expense']

df_stf = pd.read_excel(file_path['htf_stf'], sheet_name="STF",usecols=stf_columns)
df_stf = df_stf[stf_columns].set_index(['STF No', 'Line No'])
df_stf['Receipt_amount'] = df_stf['Miscellaneous Expense Incurred']+df_stf['Receipt Amount']

print(df_stf.shape)
df_stf.head(5)

(7304, 23)


Project  \
STF No     Line No                      
STF2302001 1        P.MBZ.SITEO.ITEQU   
           2        P.MBZ.SITEO.ITEQU   
           3        P.MBZ.SITEO.ITEQU   
           4        P.MBZ.SITEO.ITEQU   
           5        P.MBZ.SITEO.ITEQU   

                                                            Item name  \
STF No     Line No                                                      
STF2302001 1        Computer Notebook Lenovo Think pad E15 21E600B...   
           2        Computer HP  Elite 800-G9 SFF-I7-12700-16GB-51...   
           3               Desktop Lenovo P360 TWR I9-Win11-64 Gb-1TB   
           4                          Card Graphic Card P1000 PNY 4GB   
           5                                  Monitor LED 24" HP P24V   

                   Buyer group           Requester   STF Date Delivery date  \
STF No     Line No                                                            
STF2302001 1            NECATI  RASİM VOLKAN SAVGA 2023-02-27    2023-02-27   
           2            NECATI  RASİM VOLKAN SAVGA 2023-02-27    2023-02-27   
           3            NECATI  RASİM VOLKAN SAVGA 2023-02-27    2023-02-27   
           4            NECATI  RASİM VOLKAN SAVGA 2023-02-27    2023-02-27   
           5            NECATI  RASİM VOLKAN SAVGA 2023-02-27    2023-02-27   

                      Status   Unit  STF qty  Received Qty  ...  Currency  \
STF No     Line No                                          ...             
STF2302001 1        Invoiced  Piece     10.0          10.0  ...       SAR   
           2        Invoiced  Piece     24.0          24.0  ...       SAR   
           3        Approved  Piece      3.0           0.0  ...       SAR   
           4        Approved  Piece      3.0           0.0  ...       SAR   
           5        Invoiced  Piece     24.0          24.0  ...       SAR   

                    Estimated unit price  Receipt Unit price Estimate amount  \
STF No     Line No                                                             
STF2302001 1                      4185.0                 0.0         41850.0   
           2                      4050.0                 0.0         97200.0   
           3                         0.0                 0.0             0.0   
           4                         0.0                 0.0             0.0   
           5                       500.0                 0.0         12000.0   

                    Receipt Amount  Remaining amount  \
STF No     Line No                                     
STF2302001 1                   0.0               0.0   
           2                   0.0               0.0   
           3                   0.0               0.0   
           4                   0.0               0.0   
           5                   0.0               0.0   

                    Estimated Miscellaneous Expense  \
STF No     Line No                                    
STF2302001 1                                      0   
           2                                      0   
           3                                      0   
           4                                      0   
           5                                      0   

                    Miscellaneous Expense Incurred  \
STF No     Line No                                   
STF2302001 1                                   0.0   
           2                                   0.0   
           3                                   0.0   
           4                                   0.0   
           5                                   0.0   

                    The Rest Miscellaneous Expense  Receipt_amount  
STF No     Line No                                                  
STF2302001 1                                   0.0             0.0  
           2                                   0.0             0.0  
           3                                   0.0             0.0  
           4                                   0.0             0.0  
   

In [62]:
# Filter out non -amount rows from the dataframe
df1 = df_stf[((df_stf['Estimate amount']!=0)|(df_stf['Receipt_amount']!=0)|(df_stf['Remaining amount']!=0)|(df_stf['Miscellaneous Expense Incurred']!=0)|(df_stf['The Rest Miscellaneous Expense']!=0))]
print('Dataframe shape',df1.shape)

Dataframe shape (6310, 23)


In [59]:
df_stf[['Estimate amount','Receipt Amount','Remaining amount',
        'Estimated Miscellaneous Expense','Miscellaneous Expense Incurred','The Rest Miscellaneous Expense']
        ].sum()

Estimate amount                    40497387.81
Receipt Amount                     33230506.94
Remaining amount                    4346912.90
Estimated Miscellaneous Expense           0.00
Miscellaneous Expense Incurred       195325.00
The Rest Miscellaneous Expense      -195325.00
dtype: float64

In [58]:
est_columns = ['Project',
               'Item name',
               'Unit','STF qty',
               'Currency','Estimated unit price',
               'Estimate amount','Estimated Miscellaneous Expense'
               ]
df1[est_columns][['Estimate amount','Estimated Miscellaneous Expense']].sum()

Estimate amount                    32667388.68
Estimated Miscellaneous Expense           0.00
dtype: float64

In [27]:
# unquie value for the dates
for i,c_type in enumerate(df_stf.dtypes):
    if c_type == str('datetime64[ns]'):
        df_stf.groupby(df_stf.columns[i].d)
        print(df_stf.columns[i],'Nunique',df_stf.iloc[:,i].nunique())
    

STF Date Nunique 298
Delivery date Nunique 305


In [386]:
df_stf['İade Edilen Miktar'].notnull().value_counts().iloc[1]

np.int64(307)

In [394]:

for i,col in enumerate(df_stf.columns):
    
    try:
        c  = 7304 - df_stf[col].notnull().value_counts().iloc[1]
        
    except:
        c = 7304
    u_c  = len(pd.unique( df_stf[col]))
    d_type = df_stf[col].dtype
    print(f'{i:>3} {col:<25} {c:>4} {u_c:>4} {d_type}')   
    

  0 Project/Site/Dept         7304   22 object
  1 STF No                    7304  811 object
  2 STF Date                  7304  298 datetime64[ns]
  3 Buyer group               7304    8 object
  4 Talep Eden                6962   49 object
  5 Line No                   7304  309 int64
  6 Project                   7304  130 object
  7 Status                    7304    6 object
  8 Remain status             7304    2 object
  9 Delivery date             7304  307 datetime64[ns]
 10 Item number               7304 3605 object
 11 Item name                 7304 3596 object
 12 Item name.1               7295 3564 object
 13 Line description          7304 3604 object
 14 Açıklama / Kullanım Yeri  6607  322 object
 15 Warehouse                 7303    4 object
 16 Location                  7303   17 object
 17 STF qty                   7304  444 object
 18 Unit                      7304   28 object
 19 Estimated unit price      7304 1313 float64
 20 Tahmini Sair Gider        7304    2 obje